In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
data_path = Path("/kaggle/input/cs46fall22/data")
from datetime import datetime
fmt = "%Y-%m-%d %H:%M:%S"

In [4]:
df_train = pd.read_csv(data_path / "train.csv")
df_test = pd.read_csv(data_path / "test.csv")

FileNotFoundError: [Errno 2] No such file or directory: '\\kaggle\\input\\cs46fall22\\data\\train.csv'

In [ ]:
df_train = df_train[df_train.side != " "] # only keep rows with side in [L,R]

In [ ]:
df_train.head()

Next, we transform some features to integer values

In [ ]:
side_to_int = {k:i for i,k in enumerate(df_train.side.unique())}
state_to_int = {k:i for i,k in enumerate(df_train.state.unique())}

In [ ]:
# side_to_int, state_to_int
# inplace argument change the values directly
#+without creating a copy
df_train.replace({"side": side_to_int,
                  "state": state_to_int}, inplace=True)
df_test.replace({"side": side_to_int,
                  "state": state_to_int}, inplace=True)

In [ ]:
df_train.start = df_train.start.apply(lambda x:datetime.strptime(x, fmt))
df_train.end = df_train.end.apply(lambda x:datetime.strptime(x, fmt))
df_test.time = df_test.time.apply(lambda x:datetime.strptime(x, fmt))

In [ ]:
df_train.head()

In [ ]:
df_test.head()

We need to create a time feature for `df_train`. We sample a time between start and end

In [ ]:
def sample_time(start, end):
    # Sampled time is between [0.25 * duration , 0.75 duration]
    # You can change it
    delta = (end-start) * (0.25 + np.random.uniform()/2)
    return start + delta

In [ ]:
df_train.time = df_train[["start", "end"]].apply(lambda x:sample_time(x['start'], x['end']), axis=1)

In [ ]:
df_train.head()

Now we can add some new features based on `time`

In [ ]:
for df in [df_train, df_test]:
    df["day"] = df.time.map(lambda x: x.day)
    df["month"] = df.time.map(lambda x: x.month)
    df["year"] = df.time.map(lambda x: x.year)
    df["hour"] = df.time.map(lambda x: x.hour)
    df["weekday"] = df.time.map(lambda x: x.weekday())

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
X_train = df_train[["lat","lng", "side", "state",
                    "day", "month", "year", "hour", "weekday"]]
X_test = df_test[["lat","lng", "side", "state",
                    "day", "month", "year", "hour", "weekday"]]
y_train = df_train["event"]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler().fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)